In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly"


In [2]:
import pkgutil

from io import BytesIO

from pandas import Series

from pandas import read_csv

from pandas import to_datetime

def read_file(fname):
    df = read_csv("./pvlib/" + fname, names=["Timestamp", "Power"])
    df["Timestamp"] = to_datetime(df["Timestamp"], format="%Y-%m-%dT%H:%M:%S%z", utc=True)
    s = df.set_index("Timestamp")["Power"]
    s = s.asfreq("H")
    return s.tz_convert("Europe/Madrid")


generation = read_file("data/generated.csv")
load = read_file("data/consumed.csv")

In [6]:
from pvlib.powerflow import self_consumption_ac_battery_custom_dispatch
from pvlib.battery import fit_sam
from pvlib.battery import sam
from pvlib.powerflow import self_consumption

from pvlib.inverter import sandia_multi

In [42]:
import pvlib
from pvlib.location import Location
from pvlib.modelchain import ModelChain
from pvlib.pvsystem import PVSystem, Array, FixedMount


def run_model_chain():
    name = 'Madrid'
    latitude = 40.31672645215922
    longitude = -3.674695061062714
    altitude = 603
    timezone = 'Europe/Madrid'
    
    module = pvlib.pvsystem.retrieve_sam('SandiaMod')['Canadian_Solar_CS5P_220M___2009_']
    inverter = pvlib.pvsystem.retrieve_sam('cecinverter')['Shenzhen_Growatt_New_Energy_Technology_Co___Ltd__Growatt_3000HF_US__240V_']

    weather = pvlib.iotools.get_pvgis_tmy(latitude, longitude, map_variables=True)[0]
    weather.index = load.index
    weather.index.name = "Timestamp"

    location = Location(
        latitude,
        longitude,
        name=name,
        altitude=altitude,
        tz=timezone,
    )
    mount = FixedMount(surface_tilt=latitude, surface_azimuth=180)
    temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']
    array = Array(
        mount=mount,
        module_parameters=module,
        modules_per_string=8,
        strings=1,
        temperature_model_parameters=temperature_model_parameters,
    )
    system = PVSystem(arrays=[array], inverter_parameters=inverter)
    mc = ModelChain(system, location)
    mc.run_model(weather)
    return mc

In [43]:
mc = run_model_chain()
self_consumption_flow = self_consumption(mc.results.ac, load)
self_consumption_flow.groupby(self_consumption_flow.index.hour).mean()[["System to load", "Grid to load", "System to grid"]].plot.bar(title="Average power flow").show()

In [35]:
battery_datasheet = {                                                                   
    "brand": "Sonnen",
    "model": "sonnenBatterie 10/5,5",
    "width": 0.690,
    "height": 1.840,
    "depth": 0.270,
    "weight": 98,
    "chemistry": "LFP",
    "mode": "AC",
    "charge_efficiency": 0.96,
    "discharge_efficiency": 0.96,
    "min_soc_percent": 5,
    "max_soc_percent": 95,
    "dc_modules": 1,
    "dc_modules_in_series": 1,
    "dc_energy_wh": 5500,
    "dc_nominal_voltage": 102.4,
    "dc_max_power_w": 3400,
}

In [112]:
from pvlib.inverter import _sandia_eff
from pvlib.inverter import _sandia_limits
import numpy as np

from pvlib.battery import sam

def sandia_multi_dc_battery(v_dc, p_dc, inverter, dispatch, battery, model):                   
    power_dc = sum(p_dc)                                                                                                                  

    # First, limit charging to the available DC power
    max_charging = -power_dc
    charging_mask = dispatch < 0
    dispatch[charging_mask] = np.max([dispatch, max_charging], axis=0)[charging_mask]

    # Second, limit discharging to the inverter's maximum output power (approximately)
    # Note this can revert the dispatch and charge when there is too much DC power (prevents clipping)
    max_discharging = inverter['Paco'] - power_dc
    discharging_mask = dispatch > 0
    dispatch[discharging_mask] = np.min([dispatch, max_discharging], axis=0)[discharging_mask]

    # Calculate the actual battery power flow
    final_state, results = model(battery, dispatch)
    charge = -results['Power'].copy()
    charge.loc[charge < 0] = 0
    discharge = results['Power'].copy()
    discharge.loc[discharge < 0] = 0

    # Adjust the DC power
    ratios = [sum(power) / sum(power_dc) for power in p_dc]
    adjusted_p_dc = [power - ratio * charge for (power, ratio) in zip(p_dc, ratios)]
    final_dc_power = sum(adjusted_p_dc) + discharge

    pv_ac_power = 0. * final_dc_power
    for vdc, pdc in zip(v_dc, adjusted_p_dc):
        pv_ac_power += pdc / final_dc_power * _sandia_eff(vdc, final_dc_power, inverter)

    vdc = inverter["Vdcmax"] / 2
    pdc = discharge
    battery_ac_power = pdc / final_dc_power * _sandia_eff(vdc, final_dc_power, inverter)

    total_ac_power = pv_ac_power + battery_ac_power

    ac_power = _sandia_limits(total_ac_power, final_dc_power, inverter['Paco'],                     
                          inverter['Pnt'], inverter['Pso'])
    
    battery_factor = battery_ac_power / ac_power
    return ac_power, battery_factor


mc = run_model_chain()
dispatch = self_consumption_flow["Grid to load"] - self_consumption_flow["System to grid"]
print(dispatch.sum())

ac_power, battery_factor = sandia_multi_dc_battery(
    [mc.results.dc["v_mp"]],
    [mc.results.dc["p_mp"]],
    mc.system.inverter_parameters,
    dispatch,
    pvlib.battery.fit_sam(battery_datasheet),
    pvlib.battery.sam,
)
dc_battery_flow = self_consumption(ac_power, load)
dc_battery_flow["Battery to load"] = dc_battery_flow["System to load"] * battery_factor
dc_battery_flow["System to load"] *= (1 - battery_factor)
dc_battery_flow.groupby(dc_battery_flow.index.hour).mean()[["System to load", "Battery to load", "Grid to load", "System to grid"]].plot.bar(title="Average power flow").show()

2929580.5103432657


In [81]:
from pvlib.powerflow import self_consumption_ac_battery_custom_dispatch

battery = fit_sam(battery_datasheet)
state, flow = self_consumption_ac_battery_custom_dispatch(self_consumption_flow, dispatch, battery, sam)
flow.groupby(flow.index.hour).mean()[["System to load", "Battery to load", "Grid to load", "System to grid"]].plot.bar(title="Average power flow").show()